In [4]:
import sqlite3
import pandas as pd
import json
import gzip
import zlib
import pickle
import base64
from datetime import datetime

def decode_blob(blob_data):
    """
    Try to decode BLOB data using various methods
    """
    if blob_data is None:
        return "NULL"
    
    if isinstance(blob_data, str):
        return blob_data
    
    # Try different decoding methods
    methods = []
    
    # 1. Try as UTF-8 string
    try:
        decoded = blob_data.decode('utf-8')
        methods.append(("UTF-8 String", decoded))
    except:
        pass
    
    # 2. Try as gzip compressed
    try:
        decompressed = gzip.decompress(blob_data)
        try:
            decoded = decompressed.decode('utf-8')
            methods.append(("Gzip + UTF-8", decoded))
        except:
            methods.append(("Gzip (binary)", f"<{len(decompressed)} bytes>"))
    except:
        pass
    
    # 3. Try as zlib compressed
    try:
        decompressed = zlib.decompress(blob_data)
        try:
            decoded = decompressed.decode('utf-8')
            methods.append(("Zlib + UTF-8", decoded))
        except:
            methods.append(("Zlib (binary)", f"<{len(decompressed)} bytes>"))
    except:
        pass
    
    # 4. Try as base64
    try:
        decoded = base64.b64decode(blob_data).decode('utf-8')
        methods.append(("Base64", decoded))
    except:
        pass
    
    # 5. Try as pickled object
    try:
        unpickled = pickle.loads(blob_data)
        methods.append(("Pickle", str(unpickled)))
    except:
        pass
    
    # 6. Try as JSON (after UTF-8 decode)
    for method_name, decoded_str in methods:
        if isinstance(decoded_str, str):
            try:
                json_obj = json.loads(decoded_str)
                return f"JSON: {json.dumps(json_obj, indent=2)[:500]}{'...' if len(str(json_obj)) > 500 else ''}"
            except:
                pass
    
    # Return the first successful decode or raw info
    if methods:
        method_name, decoded = methods[0]
        if isinstance(decoded, str) and len(decoded) > 500:
            return f"{method_name}: {decoded[:500]}..."
        return f"{method_name}: {decoded}"
    else:
        return f"<Binary data: {len(blob_data)} bytes>"

def inspect_sqlite_db(db_path):
    """
    Inspect a SQLite database and show its contents
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        print(f"=== Inspecting SQLite Database: {db_path} ===\n")
        
        # 1. List all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print("📋 Tables found:")
        for table in tables:
            print(f"  - {table[0]}")
        print()
        
        # 2. For each table, show structure and sample data
        for table_name in [t[0] for t in tables]:
            print(f"🔍 Table: {table_name}")
            print("-" * 50)
            
            # Show table schema
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            
            print("📊 Schema:")
            for col in columns:
                col_id, name, data_type, not_null, default, pk = col
                nullable = "NOT NULL" if not_null else "NULL"
                primary = "PRIMARY KEY" if pk else ""
                print(f"  {name} ({data_type}) {nullable} {primary}")
            
            # Count rows
            cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
            row_count = cursor.fetchone()[0]
            print(f"\n📈 Row count: {row_count}")
            
            # Show sample data with BLOB decoding
            if row_count > 0:
                print(f"\n📋 Sample data (first 5 rows):")
                cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
                rows = cursor.fetchall()
                
                # Get column names
                column_names = [desc[0] for desc in cursor.description]
                
                # Process each row
                for i, row in enumerate(rows):
                    print(f"\n--- Row {i+1} ---")
                    for col_name, value in zip(column_names, row):
                        # Check if this column is BLOB type
                        col_info = next((c for c in columns if c[1] == col_name), None)
                        if col_info and col_info[2].upper() in ['BLOB', 'BINARY']:
                            decoded_value = decode_blob(value)
                            print(f"{col_name}: {decoded_value}")
                        elif col_name == 'expires' and isinstance(value, (int, float)):
                            # Convert timestamp to readable date
                            try:
                                readable_date = datetime.fromtimestamp(value).strftime('%Y-%m-%d %H:%M:%S')
                                print(f"{col_name}: {value} ({readable_date})")
                            except:
                                print(f"{col_name}: {value}")
                        else:
                            print(f"{col_name}: {value}")
            else:
                print("\n(No data in this table)")
            
            print("\n" + "="*70 + "\n")
        
        conn.close()
        
    except sqlite3.Error as e:
        print(f"❌ SQLite error: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")

def inspect_specific_key(db_path, key_value, table_name="responses"):
    """
    Inspect a specific cache entry by key
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute(f"SELECT * FROM {table_name} WHERE key = ?", (key_value,))
        row = cursor.fetchone()
        
        if row:
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            column_names = [col[1] for col in columns]
            
            print(f"🔍 Entry for key: '{key_value}'")
            print("-" * 50)
            
            for col_name, value in zip(column_names, row):
                col_info = next((c for c in columns if c[1] == col_name), None)
                if col_info and col_info[2].upper() in ['BLOB', 'BINARY']:
                    decoded_value = decode_blob(value)
                    print(f"\n{col_name}:")
                    print(decoded_value)
                elif col_name == 'expires' and isinstance(value, (int, float)):
                    try:
                        readable_date = datetime.fromtimestamp(value).strftime('%Y-%m-%d %H:%M:%S')
                        print(f"{col_name}: {value} ({readable_date})")
                    except:
                        print(f"{col_name}: {value}")
                else:
                    print(f"{col_name}: {value}")
        else:
            print(f"❌ No entry found for key: '{key_value}'")
        
        conn.close()
        
    except Exception as e:
        print(f"❌ Error: {e}")

def list_all_keys(db_path, table_name="responses", limit=20):
    """
    List all keys in the cache
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute(f"SELECT key, expires FROM {table_name} ORDER BY key LIMIT {limit};")
        rows = cursor.fetchall()
        
        print(f"📋 Cache keys (showing first {limit}):")
        print("-" * 50)
        
        for key, expires in rows:
            if expires and isinstance(expires, (int, float)):
                try:
                    readable_date = datetime.fromtimestamp(expires).strftime('%Y-%m-%d %H:%M:%S')
                    expiry_info = f" (expires: {readable_date})"
                except:
                    expiry_info = f" (expires: {expires})"
            else:
                expiry_info = ""
            
            print(f"  {key}{expiry_info}")
        
        conn.close()
        
    except Exception as e:
        print(f"❌ Error: {e}")

def query_table(db_path, table_name, limit=10):
    """
    Query a specific table and return results as DataFrame
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT {limit};", conn)
        conn.close()
        return df
    except Exception as e:
        print(f"❌ Error querying table: {e}")
        return None

def run_custom_query(db_path, query):
    """
    Run a custom SQL query
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"❌ Error running query: {e}")
        return None

# Main inspection
if __name__ == "__main__":
    db_file = "/home/thiesen/Documents/AI-Innoscence_Ecosystem/hamburg_branchenbuch_cache.sqlite"
    
    # 1. Inspect the entire database
    inspect_sqlite_db(db_file)
    
    # 2. List all cache keys
    print("\n" + "="*70)
    list_all_keys(db_file, limit=10)
    
    # 3. Inspect a specific key (uncomment and replace with actual key)
    # inspect_specific_key(db_file, "your_specific_key_here")
    
    # 4. Example: Query specific entries
    # result = run_custom_query(db_file, "SELECT key, LENGTH(value) as value_size FROM responses ORDER BY value_size DESC LIMIT 5")
    # print("\nLargest cache entries:")
    # print(result)
    
    print("\n🔧 Usage examples:")
    print("- inspect_specific_key(db_file, 'some_key') - View a specific cache entry")
    print("- list_all_keys(db_file, limit=50) - List more keys")
    print("- run_custom_query(db_file, 'SELECT COUNT(*) FROM responses') - Custom SQL")

=== Inspecting SQLite Database: /home/thiesen/Documents/AI-Innoscence_Ecosystem/hamburg_branchenbuch_cache.sqlite ===

📋 Tables found:
  - responses
  - redirects

🔍 Table: responses
--------------------------------------------------
📊 Schema:
  key (TEXT) NULL PRIMARY KEY
  value (BLOB) NULL 
  expires (INTEGER) NULL 

📈 Row count: 288

📋 Sample data (first 5 rows):

--- Row 1 ---
key: 1385678bff52768e
value: Pickle: {'_content': b'\n\n\n  \n\n\n<html class="video no-touchevents k-widget-bo-page js-priorityNav no-touchevents" lang="de">\n   <head>\n      <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n      <meta charset="utf-8" />\n      <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n      <script type="text/javascript">\n      \t\twindow.JS_LANG=\'de\';\n      \t\twindow.hhChat = { \'chatUrl\' : \'\' };\n      </script> \n      <meta name="viewport" content="width=device-wi...
expires: 1758883924 (2025-09-26 12:52:04)

--- Row 2 ---
key: a76278384f88